# SQL to Python MySQL Script

Este notebook executa comandos SQL em um banco de dados MySQL usando `mysql-connector-python`.

In [ ]:

# Instale a biblioteca necessária, se ainda não tiver feito isso
# !pip install mysql-connector-python
import mysql.connector


## Configuração da Conexão com o Banco de Dados

In [ ]:

# Configurações de conexão
db_config = {
    'user': 'seu_usuario',  # Substitua por seu usuário do MySQL
    'password': 'sua_senha',  # Substitua por sua senha do MySQL
    'host': 'localhost',  # Substitua por seu host (localhost se for local)
    'database': 'mydb'  # O banco de dados a ser usado
}

# Conectar ao banco de dados
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()


## Desativando verificações temporariamente

In [ ]:

# Desativando checagem de constraints temporariamente
cursor.execute("SET @OLD_UNIQUE_CHECKS=@@UNIQUE_CHECKS, UNIQUE_CHECKS=0;")
cursor.execute("SET @OLD_FOREIGN_KEY_CHECKS=@@FOREIGN_KEY_CHECKS, FOREIGN_KEY_CHECKS=0;")
cursor.execute("SET @OLD_SQL_MODE=@@SQL_MODE, SQL_MODE='TRADITIONAL,ALLOW_INVALID_DATES';")


## Criando o Schema e as Tabelas

In [ ]:

# Criar schema
cursor.execute("CREATE SCHEMA IF NOT EXISTS `mydb` DEFAULT CHARACTER SET utf8 COLLATE utf8_general_ci;")
cursor.execute("USE `mydb`;")

# Criar tabela usuario
cursor.execute("""
CREATE TABLE IF NOT EXISTS `usuario` (
    `cpf` BIGINT NOT NULL,
    `nome` VARCHAR(200) NULL,
    `dtnascimento` DATE NULL,
    `email` VARCHAR(100) NULL,
    `senhahash` VARCHAR(30) NULL,
    `dtcadastro` DATE NULL,
    `classificacao` CHAR(1) NULL,
    PRIMARY KEY (`cpf`)
) ENGINE = InnoDB;
""")

# Criar tabela endereco
cursor.execute("""
CREATE TABLE IF NOT EXISTS `endereco` (
    `idendereco` INT NOT NULL,
    `rua` VARCHAR(200) NULL,
    `cidade` VARCHAR(100) NULL,
    `estado` VARCHAR(50) NULL,
    `cep` VARCHAR(45) NULL,
    `pais` VARCHAR(45) NULL,
    `enderecocol` VARCHAR(45) NULL,
    `usuario_cpf` BIGINT NULL,
    PRIMARY KEY (`idendereco`),
    INDEX `usuario_cpf_idx` (`usuario_cpf` ASC),
    CONSTRAINT `usuario_cpf`
    FOREIGN KEY (`usuario_cpf`)
    REFERENCES `usuario` (`cpf`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION
) ENGINE = InnoDB;
""")


## Continuando a criação das tabelas...

In [ ]:

# Criar tabela categoriaproduto
cursor.execute("""
CREATE TABLE IF NOT EXISTS `categoriaproduto` (
    `idcategoria` INT NOT NULL,
    `nome` VARCHAR(100) NULL,
    `descricao` VARCHAR(500) NULL,
    PRIMARY KEY (`idcategoria`)
) ENGINE = InnoDB;
""")

# Criar tabela logista
cursor.execute("""
CREATE TABLE IF NOT EXISTS `logista` (
    `cnpj` VARCHAR(30) NOT NULL,
    `razaosocial` VARCHAR(100) NULL,
    `telefone` VARCHAR(45) NULL,
    PRIMARY KEY (`cnpj`)
) ENGINE = InnoDB;
""")


## Finalizando e Restaurando Configurações

In [ ]:

# Restaurando as configurações antigas
cursor.execute("SET SQL_MODE=@OLD_SQL_MODE;")
cursor.execute("SET FOREIGN_KEY_CHECKS=@OLD_FOREIGN_KEY_CHECKS;")
cursor.execute("SET UNIQUE_CHECKS=@OLD_UNIQUE_CHECKS;")

# Fechar conexão
cursor.close()
connection.close()

print("Script SQL executado com sucesso!")


In [ ]:

# Criar tabela produto
cursor.execute(""" 
CREATE TABLE IF NOT EXISTS `produto` (
    `idproduto` INT NOT NULL,
    `categoria_idcategoria` INT NOT NULL,
    `logista_cnpj` VARCHAR(30) NULL,
    `nomeproduto` VARCHAR(300) NULL,
    `descricao` VARCHAR(300) NULL,
    `qtestoque` INT NULL,
    `preco` FLOAT NULL,
    PRIMARY KEY (`idproduto`),
    INDEX `categoria_idcategoria_idx` (`categoria_idcategoria` ASC),
    INDEX `logista_cnpj_idx` (`logista_cnpj` ASC),
    CONSTRAINT `categoria_idcategoria`
    FOREIGN KEY (`categoria_idcategoria`)
    REFERENCES `categoriaproduto` (`idcategoria`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
    CONSTRAINT `logista_cnpj`
    FOREIGN KEY (`logista_cnpj`)
    REFERENCES `logista` (`cnpj`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION
) ENGINE = InnoDB;
""")


In [ ]:

# Criar tabela avaliacao
cursor.execute(""" 
CREATE TABLE IF NOT EXISTS `avaliacao` (
    `idavaliacao` INT NOT NULL,
    `usuario_cpf` BIGINT NOT NULL,
    `idproduto` INT NOT NULL,
    `estrelas` INT NULL,
    `comentario` VARCHAR(500) NULL,
    `dtavaliacao` DATETIME NULL,
    PRIMARY KEY (`idavaliacao`),
    INDEX `usuario_cpf_idx` (`usuario_cpf` ASC),
    INDEX `produto_idproduto_idx` (`idproduto` ASC),
    CONSTRAINT `usuario_cpf`
    FOREIGN KEY (`usuario_cpf`)
    REFERENCES `usuario` (`cpf`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
    CONSTRAINT `produto_idproduto`
    FOREIGN KEY (`idproduto`)
    REFERENCES `produto` (`idproduto`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION
) ENGINE = InnoDB;
""")


In [ ]:

# Criar tabela carrinho
cursor.execute(""" 
CREATE TABLE IF NOT EXISTS `carrinho` (
    `idcarinho` INT NOT NULL,
    `usuario_cpf` BIGINT NOT NULL,
    `dtvisita` DATETIME NULL,
    `pagina` VARCHAR(45) NULL COMMENT 'URL ou identificador da página vistiada',
    PRIMARY KEY (`idcarinho`),
    INDEX `usuario_cpf_idx` (`usuario_cpf` ASC),
    CONSTRAINT `usuario_cpf`
    FOREIGN KEY (`usuario_cpf`)
    REFERENCES `usuario` (`cpf`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION
) ENGINE = InnoDB;
""")


In [ ]:

# Criar tabela itenscarrinho
cursor.execute(""" 
CREATE TABLE IF NOT EXISTS `itenscarrinho` (
    `iditenscarrinho` INT NOT NULL,
    `carrinho_idcarinho` INT NOT NULL,
    `produto_idproduto` INT NOT NULL,
    `qtitens` INT NULL,
    PRIMARY KEY (`iditenscarrinho`),
    INDEX `carinho_idcarinho_idx` (`carrinho_idcarinho` ASC),
    INDEX `porduto_idproduto_idx` (`produto_idproduto` ASC),
    CONSTRAINT `carrinho_idcarinho`
    FOREIGN KEY (`carrinho_idcarinho`)
    REFERENCES `carrinho` (`idcarinho`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
    CONSTRAINT `porduto_idproduto`
    FOREIGN KEY (`produto_idproduto`)
    REFERENCES `produto` (`idproduto`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION
) ENGINE = InnoDB;
""")
